In [74]:
!pip install tensorflow==2.17 --quiet

In [2]:
import tensorflow as tf
print(tf.__version__)

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

import random, os, sys
from glob import glob
import numpy as np

2.17.0
Num GPUs Available:  1


In [3]:
def average_wavelengths(chip, wavelength_indices):
    out_chip = chip[:,:,wavelength_indices]
    out_chip = np.nanmean(out_chip, axis=2)
    return out_chip

def convert_chip_to_different_spectra(chip, in_wavelengths, out_wavelengths):
    out_chip = []
    for out_wavelength_index in range(len(out_wavelengths)):
        out_wavelength_start = out_wavelengths[out_wavelength_index][0]
        out_wavelength_end = out_wavelengths[out_wavelength_index][1]
        
        in_wavelengths_to_grab = []
        for i in range(len(in_wavelengths)):
            in_wavelength = in_wavelengths[i]
            if in_wavelength > out_wavelength_start and in_wavelength < out_wavelength_end:
                in_wavelengths_to_grab.append(i)
        
        new_chip_at_wavelegnth = average_wavelengths(chip, in_wavelengths_to_grab)
        #print(new_chip_at_wavelegnth.shape)
        out_chip.append(new_chip_at_wavelegnth)
        
    out_chip = np.asarray(out_chip)
    out_chip = np.moveaxis(out_chip,0,2)
    #print(out_chip.shape)
    return out_chip

def convert_emit_chip_to_planetscope_chip(emit_chip_filename, planetscope_chip_filename, emit_wavelengths, planetscope_wavelengths):
    in_chip = np.load(emit_chip_filename)
    out_chip = convert_chip_to_different_spectra(in_chip, emit_wavelengths, planetscope_wavelengths)
    np.save(planetscope_chip_filename, out_chip)
    
def convert_whole_dataset(emit_chip_directory, planetscope_chip_directory, emit_wavelengths, planetscope_wavelengths, chipname_prefix = "chip_"):
    if not os.path.exists(planetscope_chip_directory):
        os.mkdir(planetscope_chip_directory)
        
    in_chip_paths = glob(os.path.join(emit_chip_directory,'*'))
    for chip_index, in_chip_path in enumerate(in_chip_paths):
        out_filename = chipname_prefix + f"{chip_index:05}" + ".npy"
        out_path = os.path.join(planetscope_chip_directory, out_filename)
        
        convert_emit_chip_to_planetscope_chip(in_chip_path, out_path, emit_wavelengths, planetscope_wavelengths)
            
        
        
#Src: https://assets.planet.com/docs/Planet_PSScene_Imagery_Product_Spec_letter_screen.pdf Pg 10
planetscope_old_wavelengths = [(455, 515), (500, 590), (590, 670), (780,860)]
planetscope_new_wavelengths = [(431,452), (465,515), (513,549), (547,583), (600,620), (650,680), (697,713), (845,885)]
        
emit_wavelengths = [ 381.00558,  388.4092 ,  395.81583,  403.2254 ,  410.638  ,
        418.0536 ,  425.47214,  432.8927 ,  440.31726,  447.7428 ,
        455.17035,  462.59888,  470.0304 ,  477.46292,  484.89743,
        492.33292,  499.77142,  507.2099 ,  514.6504 ,  522.0909 ,
        529.5333 ,  536.9768 ,  544.42126,  551.8667 ,  559.3142 ,
        566.7616 ,  574.20905,  581.6585 ,  589.108  ,  596.55835,
        604.0098 ,  611.4622 ,  618.9146 ,  626.36804,  633.8215 ,
        641.2759 ,  648.7303 ,  656.1857 ,  663.6411 ,  671.09753,
        678.5539 ,  686.0103 ,  693.4677 ,  700.9251 ,  708.38354,
        715.84094,  723.2993 ,  730.7587 ,  738.2171 ,  745.6765 ,
        753.1359 ,  760.5963 ,  768.0557 ,  775.5161 ,  782.97754,
        790.4379 ,  797.89935,  805.36176,  812.8232 ,  820.2846 ,
        827.746  ,  835.2074 ,  842.66986,  850.1313 ,  857.5937 ,
        865.0551 ,  872.5176 ,  879.98004,  887.44147,  894.90393,
        902.3664 ,  909.82886,  917.2913 ,  924.7538 ,  932.21625,
        939.6788 ,  947.14026,  954.6027 ,  962.0643 ,  969.5268 ,
        976.9883 ,  984.4498 ,  991.9114 ,  999.37286, 1006.8344 ,
       1014.295  , 1021.7566 , 1029.2172 , 1036.6777 , 1044.1383 ,
       1051.5989 , 1059.0596 , 1066.5201 , 1073.9797 , 1081.4404 ,
       1088.9    , 1096.3597 , 1103.8184 , 1111.2781 , 1118.7368 ,
       1126.1964 , 1133.6552 , 1141.1129 , 1148.5717 , 1156.0304 ,
       1163.4882 , 1170.9459 , 1178.4037 , 1185.8616 , 1193.3184 ,
       1200.7761 , 1208.233  , 1215.6898 , 1223.1467 , 1230.6036 ,
       1238.0596 , 1245.5154 , 1252.9724 , 1260.4283 , 1267.8833 ,
       1275.3392 , 1282.7942 , 1290.2502 , 1297.7052 , 1305.1603 ,
       1312.6144 , 1320.0685 , 1327.5225 , 1334.9756 , 1342.4287 ,
       1349.8818 , 1357.3351 , 1364.7872 , 1372.2384 , 1379.6907 ,
       1387.1418 , 1394.5931 , 1402.0433 , 1409.4937 , 1416.944  ,
       1424.3933 , 1431.8427 , 1439.292  , 1446.7404 , 1454.1888 ,
       1461.6372 , 1469.0847 , 1476.5321 , 1483.9796 , 1491.4261 ,
       1498.8727 , 1506.3192 , 1513.7649 , 1521.2104 , 1528.655  ,
       1536.1007 , 1543.5454 , 1550.9891 , 1558.4329 , 1565.8766 ,
       1573.3193 , 1580.7621 , 1588.205  , 1595.6467 , 1603.0886 ,
       1610.5295 , 1617.9705 , 1625.4104 , 1632.8513 , 1640.2903 ,
       1647.7303 , 1655.1694 , 1662.6074 , 1670.0455 , 1677.4836 ,
       1684.9209 , 1692.358  , 1699.7952 , 1707.2314 , 1714.6667 ,
       1722.103  , 1729.5383 , 1736.9727 , 1744.4071 , 1751.8414 ,
       1759.2749 , 1766.7084 , 1774.1418 , 1781.5743 , 1789.007  ,
       1796.4385 , 1803.8701 , 1811.3008 , 1818.7314 , 1826.1611 ,
       1833.591  , 1841.0206 , 1848.4495 , 1855.8773 , 1863.3052 ,
       1870.733  , 1878.16   , 1885.5869 , 1893.013  , 1900.439  ,
       1907.864  , 1915.2892 , 1922.7133 , 1930.1375 , 1937.5607 ,
       1944.9839 , 1952.4071 , 1959.8295 , 1967.2518 , 1974.6732 ,
       1982.0946 , 1989.515  , 1996.9355 , 2004.355  , 2011.7745 ,
       2019.1931 , 2026.6118 , 2034.0304 , 2041.4471 , 2048.865  ,
       2056.2808 , 2063.6965 , 2071.1123 , 2078.5273 , 2085.9421 ,
       2093.3562 , 2100.769  , 2108.1821 , 2115.5942 , 2123.0063 ,
       2130.4175 , 2137.8289 , 2145.239  , 2152.6482 , 2160.0576 ,
       2167.467  , 2174.8755 , 2182.283  , 2189.6904 , 2197.097  ,
       2204.5034 , 2211.9092 , 2219.3147 , 2226.7195 , 2234.1233 ,
       2241.5269 , 2248.9297 , 2256.3328 , 2263.7346 , 2271.1365 ,
       2278.5376 , 2285.9387 , 2293.3386 , 2300.7378 , 2308.136  ,
       2315.5342 , 2322.9326 , 2330.3298 , 2337.7263 , 2345.1216 ,
       2352.517  , 2359.9126 , 2367.3071 , 2374.7007 , 2382.0935 ,
       2389.486  , 2396.878  , 2404.2695 , 2411.6604 , 2419.0513 ,
       2426.4402 , 2433.8303 , 2441.2183 , 2448.6064 , 2455.9944 ,
       2463.3816 , 2470]
    
emit_dataset_path = "./reflectance/"
planetscope_old_dataset_path = "./dataset_for_cnn/x/"
planetscope_new_dataset_path = "./dataset_for_cnn/y/"

convert_whole_dataset(emit_dataset_path, planetscope_old_dataset_path, emit_wavelengths, planetscope_old_wavelengths, chipname_prefix = "chip_")
convert_whole_dataset(emit_dataset_path, planetscope_new_dataset_path, emit_wavelengths, planetscope_new_wavelengths, chipname_prefix = "chip_")

In [30]:
class CustomDataGenerator(tf.keras.utils.Sequence):
    
    def __init__(self, x_chip_path_dir, y_chip_path_dir, batch_size, shuffle=True):
        self.x_chip_path_dir = x_chip_path_dir
        self.y_chip_path_dir = y_chip_path_dir
        
        self.x_chip_path_list = glob(os.path.join(self.x_chip_path_dir,'*'))
        self.y_chip_path_list = glob(os.path.join(self.y_chip_path_dir,'*'))
        
        self.num_files = len(self.x_chip_path_list)
        self.index_path = list(range(self.num_files))+list(range(self.num_files))
        self.batch_size = batch_size
        self.shuffle = shuffle
    
    def on_epoch_end(self):
        if self.shuffle:
            random.shuffle(self.index_path)
    
    def __getitem__(self, index):
        X = []
        Y = []
        for i in range(self.batch_size):
            x = np.load(self.x_chip_path_list[self.index_path[index+i]])
            y = np.load(self.y_chip_path_list[self.index_path[index+i]])
            
            X.append(x)
            Y.append(y)
        
        return np.asarray(X), np.asarray(Y)
    
    def __len__(self):
        return self.num_files // self.batch_size
    
class CustomDataGenerator2(tf.keras.utils.Sequence):
    def __init__(self, x_chip_path_dir, y_chip_path_dir, batch_size, shuffle=True):
        self.x_chip_path_dir = x_chip_path_dir
        self.y_chip_path_dir = y_chip_path_dir

        # List all files in the directories
        self.x_chip_path_list = glob(os.path.join(self.x_chip_path_dir, '*'))
        self.y_chip_path_list = glob(os.path.join(self.y_chip_path_dir, '*'))

        self.num_files = len(self.x_chip_path_list)
        assert len(self.x_chip_path_list) == len(self.y_chip_path_list), \
            "Mismatch between X and Y files."

        self.index_path = np.arange(self.num_files)
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def on_epoch_end(self):
        """Shuffle data at the end of each epoch."""
        if self.shuffle:
            np.random.shuffle(self.index_path)

    def __len__(self):
        """Number of batches per epoch."""
        return int(np.floor(self.num_files / self.batch_size))

    def __getitem__(self, index):
        """Generate one batch of data."""
        # Indices for this batch
        batch_indices = self.index_path[index * self.batch_size:(index + 1) * self.batch_size]

        # Initialize arrays
        X, Y = [], []
        for idx in batch_indices:
            x = np.load(self.x_chip_path_list[idx])
            y = np.load(self.y_chip_path_list[idx])

            # Ensure shapes match model requirements
            X.append(x)
            Y.append(y)

        # Convert lists to NumPy arrays
        X = np.asarray(X, dtype=np.float32)  # Ensure compatible dtype
        Y = np.asarray(Y, dtype=np.float32)
        
        print("X shape:", X.shape)
        print("Y shape:", Y.shape)
        print("X dtype:", X.dtype)
        print("Y dtype:", Y.dtype)

        # Ensure that X and Y are 4D arrays (for Conv2D, for example)
        # X shape should be (batch_size, height, width, channels), if not already
        if X.ndim == 3:  # If it is 3D, add a channels dimension (e.g., grayscale image)
            X = np.expand_dims(X, axis=-1)  # Convert shape to (batch_size, height, width, 1)
        
        # Check for nested structures or unexpected object types inside X or Y
        print(f"X is a numpy array: {isinstance(X, np.ndarray)}")
        print(f"Y is a numpy array: {isinstance(Y, np.ndarray)}")

        # Check if the arrays are consistent and fully flattened
        try:
            X = np.array(X, dtype=np.float32)
            Y = np.array(Y, dtype=np.float32)

            # Debug: check the contents of X and Y
            print(f"X[0]: {X[0]}")
            print(f"Y[0]: {Y[0]}")
        except Exception as e:
            print(f"Error while checking array structure: {e}")

        # Try converting to TensorFlow tensors
        try:
            X_tensor = tf.convert_to_tensor(X, dtype=tf.float32)
            Y_tensor = tf.convert_to_tensor(Y, dtype=tf.float32)

            print(f"X tensor shape: {X_tensor.shape}")
            print(f"Y tensor shape: {Y_tensor.shape}")
        except ValueError as e:
            print(f"Error converting to tensor: {e}")
            print(f"X: {X}")
            print(f"Y: {Y}")
            raise e

        return X_tensor, Y_tensor

In [7]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='sigmoid', padding='same'))
model.add(layers.Conv2D(64, (3, 3), activation='sigmoid', padding='same'))
model.add(layers.Conv2D(64, (3, 3), activation='sigmoid', padding='same'))
model.add(layers.Conv2D(8, (3, 3), activation='sigmoid', padding='same'))
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [31]:
batch_size = 16
model.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredError(), metrics=['accuracy'])
training_generator = CustomDataGenerator2("./dataset_for_cnn/x/", "./dataset_for_cnn/y/", batch_size)
model.fit(training_generator, epochs = 10)

X shape: (16, 64, 64, 4)
Y shape: (16, 64, 64, 8)
X dtype: float32
Y dtype: float32
X is a numpy array: True
Y is a numpy array: True
X[0]: [[[0.1343335  0.11779986 0.10692012 0.10447407]
  [0.16078621 0.14514779 0.13559139 0.13598953]
  [0.17758329 0.16281873 0.15417627 0.15618846]
  ...
  [0.17953917 0.16528389 0.15721497 0.16042693]
  [0.18528782 0.17132063 0.16355342 0.16739573]
  [0.18443614 0.17059754 0.16261928 0.1664634 ]]

 [[0.17346197 0.15840681 0.14937691 0.15141274]
  [0.17346197 0.15840681 0.14937691 0.15141274]
  [0.18457109 0.17005615 0.16137882 0.16429764]
  ...
  [0.18776588 0.17354093 0.16565707 0.16976528]
  [0.18528782 0.17132063 0.16355342 0.16739573]
  [0.18603446 0.17209376 0.16404161 0.16814357]]

 [[0.19142002 0.17709582 0.16901529 0.17272294]
  [0.18718354 0.1726883  0.16438659 0.16777486]
  [0.17205282 0.1565205  0.14702259 0.14830409]
  ...
  [0.18776588 0.17354093 0.16565707 0.16976528]
  [0.18792027 0.17395546 0.16634634 0.1706382 ]
  [0.18603446 0.172093

ValueError: object __array__ method not producing an array

In [ ]:
!nvidia-smi